In [2]:
import numpy as np
from collections import defaultdict
import xgboost as xgb
import gtnlplib.preproc
import gtnlplib.constants


In [3]:
gtnlplib.preproc.docsToBOWs(gtnlplib.constants.TRAINKEY)
gtnlplib.preproc.docsToBOWs(gtnlplib.constants.DEVKEY)
all_tr_insts,all_dev_insts= gtnlplib.preproc.loadInstances(gtnlplib.constants.TRAINKEY, gtnlplib.constants.DEVKEY)
ac_train = gtnlplib.preproc.getAllCounts(gtnlplib.preproc.dataIterator(gtnlplib.constants.TRAINKEY))


In [75]:
word_idx = {}
for w in ac_train:
    word_idx[w] = len(word_idx)

In [76]:
def get_XY(all_instances):
    idx_dict = {'POS' : 0, 'NEG' : 1, 'NEU' : 2}
    X = np.zeros((len(all_instances), len(word_idx)))
    Y = []
    for sample_idx in range(len(all_instances)):
        sample = all_instances[sample_idx]
        feat = sample[0]
        label = sample[1]
        Y.append(idx_dict[label])
        for w in feat:
            if w in word_idx:
                X[sample_idx][word_idx[w]] = 1
    Y = np.array(Y)
    return X, Y
train_X, train_Y = get_XY(all_tr_insts)
dev_X, dev_Y = get_XY(all_dev_insts)

In [ ]:
xg_train = xgb.DMatrix( train_X, label=train_Y)
xg_dev = xgb.DMatrix(dev_X, label=dev_Y)
# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.01
param['max_depth'] = 2
param['lambda'] = 5
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 3

watchlist = [ (xg_train,'train'), (xg_dev, 'dev') ]
num_round = 10000
bst = xgb.train(param, xg_train, num_round, watchlist );
# get prediction
pred = bst.predict( xg_dev );

print ('predicting, classification error=%f' % (sum( int(pred[i]) != dev_Y[i] for i in range(len(dev_Y))) / float(len(dev_Y)) ))
